## 1.1) Fetch Img.zip file and define paths
Run provided cell code to fetch dataset from Google Drive. Then define dataset paths relative to mount point.

In [ ]:
import os

df_icrb_root = '/kaggle/working/data/Datasets/DeepFashion/In-shop Clothes Retrieval Benchmark'

if not os.path.exists(df_icrb_root):
    !conda install -y gdown
    !gdown --id "14O0JBCHEFjE4RImV3to6UWmIgQa08PrB"
    !mkdir -p "$df_icrb_root"
    !mkdir -p "$df_icrb_root"
    !mv /kaggle/working/Img.zip "$df_icrb_root"/

## 1.2) Unzip Img directory in Kaggle
By unzipping the Img_processed.zip in Kaggle before running our model we gain significant disk reading speedups.
So, the first step is to unzip images directory, and then save the image directory before proceeding.

In [ ]:
df_icrb_img_root = f'{df_icrb_root}/Img'
if not os.path.exists(df_icrb_img_root):
    # Check that zip file is already there
    assert os.path.exists(f'{df_icrb_root}/Img.zip')
    # Unzip it in Kaggle
    !unzip -q "$df_icrb_root"/Img.zip -d "$df_icrb_root"
    # Handle newly-created image directory
    assert os.path.exists(df_icrb_img_root), f'df_icrb_img_root: {df_icrb_img_root}'
    assert not os.path.exists(f'{df_icrb_img_root}/Img')
    !rm -f "$df_icrb_root"/Img.zip
    assert not os.path.exists(f'{df_icrb_root}/Img.zip')

## 1.3) Clone github repo
Clone achariso/gans-thesis repo into /content/code
 using git clone.
 For more info see: https://medium.com/@purba0101/how-to-clone-private-github-repo-in-google-colab-using-ssh-77384cfef18f

In [ ]:
repo_root = '/kaggle/working/code/gans-thesis'
if not os.path.exists(repo_root) and not os.path.exists(f'{repo_root}/requirements.txt'):
    # Check that ssh keys exist
    assert os.path.exists(f'/kaggle/input/git-keys')
    id_rsa_abs_drive = f'/kaggle/input/git-keys/id_rsa'
    id_rsa_pub_abs_drive = f'{id_rsa_abs_drive}.pub'
    assert os.path.exists(id_rsa_abs_drive)
    assert os.path.exists(id_rsa_pub_abs_drive)
    # On first run: Add ssh key in repo
    # !cat "$id_rsa_pub_abs_drive", copy & paste in repo's Deploy Keys
    # Transfer config file
    ssh_config_abs_drive = f'/kaggle/input/git-keys/config'
    assert os.path.exists(ssh_config_abs_drive)
    !mkdir -p ~/.ssh
    !cp -f "$ssh_config_abs_drive" ~/.ssh/
    # # Add github.com to known hosts
    !ssh-keyscan -t rsa github.com >> ~/.ssh/known_hosts
    # Test: !ssh -T git@github.com

    # Remove any previous attempts
    !rm -rf "$repo_root"
    !mkdir -p "$repo_root"
    # Clone repo
    !git clone git@github.com:achariso/gans-thesis.git "$repo_root"
    src_root = f'{repo_root}/src'
    !rm -rf "$repo_root"/report

## 1.4) Install pip packages
All required files are stored in a requirements.txt files at the repository's root.
Use `pip install -r requirements.txt` from inside the dir to install required packages.

In [ ]:
%cd $repo_root
!pip install -r requirements.txt

import torch
assert str(torch.__version__).startswith('1.7')

## 1.5) Add code/, */src/ to path
This is necessary in order to be able to run the modules.

In [ ]:
content_root_abs = f'{repo_root}'
src_root_abs = f'{repo_root}/src'
%env PYTHONPATH="/kaggle/lib/kagglegym:/kaggle/lib:$content_root_abs:$src_root_abs"

# 2) Test code
Test that pulled code is running by running its tests.

In [ ]:
import matplotlib.pyplot as plt

# Fix: Adjust figure size for better plotting in Colab
plt.rcParams["figure.figsize"] = (20, 20)

%run src/main.py

In [ ]:
test_root = f'{repo_root}/tests'
!python -m unittest discover -s "$test_root" -t "$test_root"